# PELS/IAS UFRJ Instagram Data Scrapper
### Este projeto é uma iniciativa da coordenação de marketing do capítulo estudantil, com o objetivo do aproveitamento de técnicas de ciência de dados para análise e métricas de publicidade e contato com o público.

Importando bibliotecas:

In [191]:
from bs4 import BeautifulSoup                         #Tratamento do texto HTML
import selenium.webdriver as webdriver                #Acesso a página web
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep                                #Criação de intervalos
import pandas as pd                                   #Métodos e classes de análise de dados

Configura acesso web e acessa página de login:

In [192]:
url = 'https://www.instagram.com/accounts/login/'       #URL do insta do PELS/IAS
#Localiza driver do navegador, no caso firefox, no dispositivo, pode ser baixado em https://github.com/mozilla/geckodriver/releases
driver = webdriver.Firefox(executable_path = 'C:\\Users\\roben\\geckodriver.exe')
driver.get(url)

Faz login:

In [193]:
sleep(3)

login = 'pelsias.ufrj'
password = 'pels2019ufrj'

logField = driver.find_element_by_name("username")
passField = driver.find_element_by_name("password")

logField.send_keys(login)
passField.send_keys(password)
sleep(3)

driver.get('https://www.instagram.com/'+login)
#driver.find_elements_by_tag_name("button")[1].click()

Acessa perfil:

In [194]:
sleep(6)
try:
    driver.find_element_by_xpath("//button[contains(@class, 'sqdOP') and contains(@class, 'L3NKy') and contains(@class, 'y3zKF')]").click()
    sleep(3)
except:
    print('oi')
    pass
try:
    driver.find_element_by_xpath("//button[contains(@class, 'aOOlW') and contains(@class, 'HoLwm')]").click()
    sleep(3)
except:
    pass
driver.find_element_by_link_text("pelsias.ufrj").click()

NoSuchElementException: Message: Unable to locate element: pelsias.ufrj


Identifica posts no corpo do HTML: (Os posts são renderizados conforme se sobe ou desce na página, portanto para ler todos, é necessário fazer a leitura conforme se desce)

In [ ]:
sleep(2)
df_posts = pd.DataFrame(columns = ['code','date','likes','text'])
lastOne = None
step = 1000
start = 0
count = 0
while True:
    sleep(3)
    insta = BeautifulSoup(driver.page_source,'html.parser')
    driver.execute_script("window.scrollTo("+str(start)+", "+str(start+step)+");")
    sleep(1)
    new_posts = driver.find_elements_by_class_name("v1Nh3")
    for element in new_posts:
        code = element.find_element_by_tag_name('a').get_attribute('href')[28:-1]
        df_posts = df_posts.append({'code':code}, ignore_index=True)
    start+=step
    if new_posts[-1] == lastOne:
        count+=1
        if count == 3:
            break
    else:
        count = 0
        lastOne = new_posts[-1]
df_posts = pd.DataFrame.drop_duplicates(df_posts)
df_posts.head(10)

In [ ]:
url = 'https://www.instagram.com/p/'
for i, row in df_posts.iterrows():
    driver.get(url+row['code'])
    sleep(2)
    try:
        content = driver.find_element_by_xpath("//div[contains(@class, 'C4VMK')]").find_elements_by_tag_name('span')[-1].get_attribute('innerHTML')
        row['text'] = BeautifulSoup(content, "lxml").text
    except:
        pass
    try:
        like_content = driver.find_element_by_xpath("//div[contains(@class, 'Nm9Fw')]")
        liked_by_x = len(like_content.find_elements_by_tag_name('span'))-1
        and_other_x_people = int(like_content.find_elements_by_tag_name('span')[-1].text)
        row['likes'] = liked_by_x + and_other_x_people
    except:
        pass
    row['date'] = driver.find_element_by_tag_name('time').get_attribute('datetime')
df_posts.head(10)